## Creating alert packets from HiTS processing

In [1]:
import os
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
import lsst.alert.packet
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, ImageNormalize)

In [2]:
! ls -lth /project/mrawls/hits2015/rerun/ | head

total 0
drwxrwsr-x 89 52919 lsst_users 4.0K Feb 25 13:08 cw_2020_02
drwxrwsr-x 89 52919 lsst_users 4.0K Jan 30 05:19 cw_2020_01
drwxrwsr-x 89 52919 lsst_users 4.0K Jan  2 07:19 cw_2019_12v2
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 20 09:47 cw_2019_12
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 22:41 coaddtest7
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 03:59 coaddtest8
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 01:03 coaddtest9
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 28 02:31 coaddtest6
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 27 23:46 coaddtest5


In [3]:
# Meredith's standard processing location
REPO = '/project/mrawls/hits2015/rerun/cw_2020_01'

In [4]:
connection = sqlite3.connect(f'{REPO}/association.db')


In [5]:
pd.read_sql_query('select count(diaSourceId) from DiaSource;', connection)


count(diaSourceId)
0              634915

In [6]:
dia_sources = pd.read_sql_query('select * from DiaSource LIMIT 30;', connection)

In [7]:
dia_sources.head()

diaSourceId  ccdVisitId         diaObjectId  ssObjectId  \
0  176486665823453487    41091504  176486665823453487           0   
1  176486665823453488    41091504  176486665823453488           0   
2  176486665823453489    41091504  176486665823453489           0   
3  176486665823453490    41091504  176486665823453490           0   
4  176486665823453491    41091504  176486665823453491           0   

   parentDiaSourceId  prv_procOrder ssObjectReassocTime   midPointTai  \
0                  0              0                None  57070.141866   
1                  0              0                None  57070.141866   
2                  0              0                None  57070.141866   
3                  0              0                None  57070.141866   
4                  0              0                None  57070.141866   

           ra raErr  ...    ixxPSF    iyyPSF    ixyPSF  extendedness  \
0  149.505905  None  ...  4.532675  3.881223  0.207817          None   
1  149.506049  None  ...  4.532675  3.881223  0.207817          None   
2  149.511464  None  ...  4.532675  3.881223  0.207817          None   
3  149.511534  None  ...  4.532675  3.881223  0.207817          None   
4  149.512223  None  ...  4.532675  3.881223  0.207817          None   

  spuriousness     flags pixelId filterName  filterId  isDipole  
0         None  25299404       0          g         1         0  
1         None  25299404       0          g         1         0  
2         None  25166152       0          g         1         0  
3         None  25168328       0          g         1         0  
4         None  41943040       0          g         1         1  

[5 rows x 111 columns]

In [8]:
dia_objects = pd.read_sql_query('select * from diaObject LIMIT 30;', connection)

In [9]:
dia_objects.head()

diaObjectId               validityStart                 validityEnd  \
0  176486665823453487  2015-02-17 03:24:17.200177  2015-02-18 03:19:20.963001   
1  176486665823453488  2015-02-17 03:24:17.200177  2015-02-19 03:14:27.983084   
2  176486665823453489  2015-02-17 03:24:17.200177  2015-02-17 05:27:34.070729   
3  176486665823453490  2015-02-17 03:24:17.200177  2015-02-17 05:27:34.070729   
4  176486665823453491  2015-02-17 03:24:17.200177  2015-02-17 05:27:34.070729   

           ra raErr      decl declErr ra_decl_Cov      radecTai  pmRa  ...  \
0  149.505905  None  3.015901    None        None  57070.141866  None  ...   
1  149.506049  None  3.018367    None        None  57070.141866  None  ...   
2  149.511464  None  2.886295    None        None  57070.141866  None  ...   
3  149.511534  None  2.883162    None        None  57070.141866  None  ...   
4  149.512223  None  2.932614    None        None  57070.141866  None  ...   

  yPSFluxPercentile75 yPSFluxPercentile95 yPSFluxMin yPSFluxMax  \
0                None                None       None       None   
1                None                None       None       None   
2                None                None       None       None   
3                None                None       None       None   
4                None                None       None       None   

  yPSFluxStetsonJ yPSFluxLinearSlope yPSFluxLinearIntercept yPSFluxMaxSlope  \
0            None               None                   None            None   
1            None               None                   None            None   
2            None               None                   None            None   
3            None               None                   None            None   
4            None               None                   None            None   

  yPSFluxErrMean parent  
0           None   None  
1           None   None  
2           None   None  
3           None   None  
4           None   None  

[5 rows x 197 columns]

In [10]:
dia_objects.columns

Index(['diaObjectId', 'validityStart', 'validityEnd', 'ra', 'raErr', 'decl',
       'declErr', 'ra_decl_Cov', 'radecTai', 'pmRa',
       ...
       'yPSFluxPercentile75', 'yPSFluxPercentile95', 'yPSFluxMin',
       'yPSFluxMax', 'yPSFluxStetsonJ', 'yPSFluxLinearSlope',
       'yPSFluxLinearIntercept', 'yPSFluxMaxSlope', 'yPSFluxErrMean',
       'parent'],
      dtype='object', length=197)

In [11]:
pd.read_sql_query('select diaSourceId, diaObjectId, count(diaObjectId) from diaSource GROUP BY diaObjectId ORDER BY count(diaObjectId) DESC LIMIT 10;', connection)

diaSourceId         diaObjectId  count(diaObjectId)
0  176492665892766091  176492665892766091                  56
1  176492674482700526  176492674482700526                  56
2  176492674482700534  176492674482700534                  56
3  176492674482700538  176492674482700538                  56
4  176492674482700543  176492674482700543                  56
5  176492674482700564  176492674482700564                  56
6  176492683072635079  176492683072635079                  56
7  176492683072635108  176492683072635108                  56
8  176492683072635132  176492683072635132                  56
9  176492687367602370  176492687367602370                  56

In [12]:
#pd.read_sql_query('select * from diaObject LIMIT 30;', connection)

In [13]:
schema = lsst.alert.packet.Schema.from_file()

In [14]:
schema.definition

{'type': 'record',
 'name': 'lsst.alert',
 'fields': [{'doc': 'unique alert identifer',
   'name': 'alertId',
   'type': 'long'},
  {'name': 'diaSource',
   'type': {'type': 'record',
    'name': 'lsst.dia.diaSourceRecord',
    'fields': [{'name': 'diaSourceId', 'type': 'long'},
     {'name': 'ccdVisitId', 'type': 'long'},
     {'default': None, 'name': 'diaObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'ssObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'parentDiaSourceId', 'type': ['null', 'long']},
     {'name': 'midPointTai', 'type': 'double'},
     {'name': 'filterName', 'type': 'string'},
     {'name': 'programId', 'type': 'int'},
     {'name': 'ra', 'type': 'double'},
     {'name': 'decl', 'type': 'double'},
     {'default': None,
      'name': 'ra_decl_Cov',
      'type': ['null',
       {'type': 'record',
        'name': 'lsst.dia.ra_decl_Cov',
        'fields': [{'name': 'raSigma', 'type': 'float'},
         {'name': 'declSigma', 'typ

In [15]:
def get_current_diaObject(diaObjectId, connection):
    df =  pd.read_sql_query(f'select * from diaObject where diaObjectId = {diaObjectId} and validityEnd IS NULL', connection)
    assert(len(df) == 1)
    return df.to_dict('records')[0]

In [16]:
def get_past_diaSources_from_diaObjectId(diaObjectId, connection):
    df =  pd.read_sql_query(f'select * from diaSource where diaObjectId = {diaObjectId}', connection)
    return df.to_dict('records')

In [30]:
#get_past_diaSources_from_diaObjectId(176492665892766091, connection)

In [18]:
ds = dia_sources.iloc[0]

In [35]:
def patch_packet(alert):
    #diaSource fixes
    alert['diaSource']['programId'] = 0
    alert['diaSource']['snr'] = 0.

    
    return alert

def make_alert_json(diaSource, connection):
    alert = {}
    alert['alertId'] = 0
    alert['diaSource'] = diaSource.to_dict()
    
    diaObjectId = diaSource['diaObjectId']
    currentDiaSourceId = diaSource['diaSourceId']
    pastDiaSources = get_past_diaSources_from_diaObjectId(diaObjectId, connection)
    #alert['prvDiaSources'] = [ds for ds in pastDiaSources if ds['diaSourceId'] != currentDiaSourceId]
    alert['prvDiaSources'] = [diaSource.to_dict(),diaSource.to_dict()]
    
    alert['prvDiaForcedSources'] = None
    alert['prvDiaNondetectionLimits'] = None
    
    diaObject = get_current_diaObject(diaObjectId, connection)
    alert['diaObject'] = diaObject
    #alert['diaObject'] = None
    
    alert['ssObject'] = None
    alert['cutoutDifference'] = None
    alert['cutoutTemplate'] = None
    
    alert = patch_packet(alert)
    return alert

# schema.validate doesn't like the nested schemas but they seem to serialize fine...
#schema.validate(make_alert_json(ds, connection))
schema.deserialize(schema.serialize(make_alert_json(ds, connection)))

ValueError: [{'diaSourceId': 176486665823453487, 'ccdVisitId': 41091504, 'diaObjectId': 176486665823453487, 'ssObjectId': 0, 'parentDiaSourceId': 0, 'prv_procOrder': 0, 'ssObjectReassocTime': None, 'midPointTai': 57070.141865742786, 'ra': 149.5059050242644, 'raErr': None, 'decl': 3.015901421494416, 'declErr': None, 'ra_decl_Cov': None, 'x': 165.0045623779297, 'xErr': None, 'y': 30.108213424682617, 'yErr': None, 'x_y_Cov': None, 'apFlux': nan, 'apFluxErr': nan, 'snr': None, 'psFlux': nan, 'psFluxErr': nan, 'psRa': None, 'psRaErr': None, 'psDecl': None, 'psDeclErr': None, 'psFlux_psRa_Cov': None, 'psFlux_psDecl_Cov': None, 'psRa_psDecl_Cov': None, 'psLnL': None, 'psChi2': None, 'psNdata': 0, 'trailFlux': None, 'trailFluxErr': None, 'trailRa': None, 'trailRaErr': None, 'trailDecl': None, 'trailDeclErr': None, 'trailLength': None, 'trailLengthErr': None, 'trailAngle': None, 'trailAngleErr': None, 'trailFlux_trailRa_Cov': None, 'trailFlux_trailDecl_Cov': None, 'trailFlux_trailLength_Cov': N

In [28]:
pastDiaSources = get_past_diaSources_from_diaObjectId(diaObjectId, connection)

NameError: name 'diaObjectId' is not defined

In [36]:
with open('/project/ebellm/sample-avro-alert/bin/tmp') as f:
    schema.retrieve_alerts(f)

RuntimeError: failed to find alert data in /project/ebellm/sample-avro-alert/bin/tmp